In [0]:
if __name__ ='__main__':
    ## 학습 과정을 볼 수 있는데 최적화된 INFO 로깅 유형
    tf.logging.set_verbosity(tf.logging.INFO) ## 텐서의 5가지 로깅 유형 중 하나 
    tf.app.run(main)
    
tf.logging.set_verbosity

## 훈련 > 평가 > 예측 세 과정을 담아내고 있음

In [0]:
import tensorflow as tf
import model as ml
import data
import numpy as np
import os
import sys

from configs import DEFINES

In [0]:
# 구글 드라이브 path 설정은 여기 해야 할 듯...

In [0]:
def main(self):
    ## DATA_OUT_PATH가 없으면 생성, 있으면 그대로 폴더 사용
    data_out_path = os.path.join(os.getcwd(), DATA_OUT_PATH)
    os.makedirs(data_out_path, exist_ok = True)
    ## 단어 사전 불러오기
    word2idx, idx2word, vocabulary_length = data.load_vocabulary()
    ## 데이터 불러오기
    train_input, train_label, eval_input, eval_label = data.load_data()
    
    ## 인코더 입력, 디코더 입력, 디코더 타깃  훈련 데이터로 만들기
    train_input_enc, train_input_enc_length = data.enc_processing(train_input, word2idx)
    train_output_dec, train_input_dec_length = data.dec_input_processing(train_label, word2idx)
    
    train_target_dec = data.dec_target_processing(train_label, word2idx)
    
    ## 인코더 입력, 디코더 입력, 디코더 타깃  검증 데이터로 만들기
    eval_input_enc, eval_input_enc_length = data.enc_processing(eval_input, word2idx)
    eval_output_dec, eval_input_dec_length = data.dec_input_processing(eval_label, word2idx)
    
    eval_target_dec = data.dec_target_processing(eval_label, word2idx)
    
    ## 체크 포인트 설정
    ## data_out 폴더에 생성될 것
    check_point_path = os.path.join(os.getcwd(), DEFINES.check_point_path)
    os.makedirs(check_point_path, exist_ok = True)
    
    
    # Estimator 객체 생성
    classifier = tf.estimator.Estimator(model_fn = ml.model, ## 모델 불러오기
                                       model_dir = DEFINES.check_point_path,
                                       params = {
                                           'hidden_size': DEFINES.hidden_size,
                                           'layer_size': DEFINES.layer_size,
                                           'learning_rate': DEFINES.learning_rate,
                                           'vocabulary_length': vocabulary_length,
                                           'embedding_size': DEFINES.embeddding_size,
                                           'embedding': DEFINES.embedding,
                                           'multilayer': DEFINES.multilayer,
                                       })
    
    
    # Estimator로 학습 진행
    ## 필요 인자: 입력 함수, 스텝 
    classifier.train(input_fn = lambda: data.train_input_fn(
    train_input_enc, train_input_dec, train_target_dec, DEFINES.batch_size),
                    steps = DEFINES.train_steps)
    
    
    ## 검증 함수 정의
    eval_result = classifier.evaluate(input_fn = lambda: data.eval_input_fn(
    eval_input_enc, eval_input_dec, eval_target_dec, DEFINES.batch_size))
    
    ## 바로 성능 확인이 가능하도록 정확도 출력 함수 작성
    print('\nEVAL set accuracy: {accuracy:0.3f}\n'.format(**eval_result))
    
    
    # 결과 예측하기
    
    predic_input_enc, predic_input_enc_length = data.enc_processing(["Education is important"],
                                                                   word2idx)
    predic_input_dec, predic_input_dec_length = data.dec_input_processing([""], word2idx)
    predic_target_dec = data.dec_target_processing([""], word2idx)
    
    predictions = classifier.predict(
        input_fn = lambda: data.eval_input_fn(predic_input_enc, predic_input_dec,
                                             predic_target_dec, DEFINES.batch_size))
    
    data.pred2string(predictions, idx2word)
    
    
    
    
## python main.py로 실행
